In [1]:
import glob

%config InlineBackend.figure_format = 'svg' 
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from skimage.morphology import skeletonize
from skimage.morphology import convex_hull_image

from plantcv.plantcv.morphology import prune

from plantcv.plantcv import fill, fill_holes
import networkx as nx
import matplotlib.pyplot as plt
from sknw import build_sknw

%config InlineBackend.figure_format = 'svg' 
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from skimage.morphology import skeletonize
from skimage.morphology import convex_hull_image

from plantcv.plantcv.morphology import prune

from plantcv.plantcv import fill, fill_holes
import networkx as nx
import matplotlib.pyplot as plt
from sknw import build_sknw

In [2]:
#поиск в глубину 
def dfs(time, circle, src, visited,height,width):
    cx = int(circle[0]) # координата x центра вершины
    cy = int(circle[1]) # координата y центра вершины
    r = int(circle[2]) # радиус вершины
    todo = [(cy,cx)]
    degree = 0 # искомая степень вершины
    while todo:
        y,x = todo.pop() # берем координаты пикселя с вершины стека

        # ходим только по белым пикселям, которые лежат в пределах картинки и не были посещены ранее в этом же dfs
        if not (0 <= y < height) or not (0 <= x < width) or (src[y,x] == 0)  or (visited[y,x] == time):
            continue

        visited[y,x] = time # помечаем пиксель как посещенный 
        dist = (cx - x)**2 + (cy - y)**2

        # если мы удалились на расстояние в 3r от центра вершины, значит нашли новое ребро
        if dist > 9 * r * r: 
            R = r

            # помечаем все пиксели в радиусе R как посещенные
            cv.circle(visited, (x,y), int(R / 3), time, -1) 
            degree+=1 
            continue
        
        for dy in range(-1, 2):
            for dx in range(-1, 2):
                ny = y + dy
                nx = x + dx
                todo += [(ny, nx)]
    #plt.imshow(visited + src)
    #plt.show()
    return degree

In [3]:
def get_description_graph(src):

    gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY) 
    detected_circles = cv.medianBlur(gray, 5) 

    circles = cv.HoughCircles(detected_circles, cv.HOUGH_GRADIENT, 1.35, minDist=75,
                               param1=120, param2=35,
                               minRadius=5, maxRadius=25)
    
    tmp = cv.GaussianBlur(gray,(15,15), 0)
    tmp = cv.adaptiveThreshold(tmp, 255,cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY,9,2)  
    tmp = (255 - tmp)
    #plt.imshow(tmp, cmap='gray')
    #plt.show()
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3,3))
    tmp = cv.erode(tmp,  kernel, iterations=2)
    #plt.imshow(tmp, cmap='gray')
    #plt.show()
    r = int(np.min(circles[0, :][-1]))
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (2 * r, 2* r))
    tmp = cv.morphologyEx(tmp, cv.MORPH_DILATE, kernel)
    #plt.imshow(tmp, cmap='gray')
    #plt.show()
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3,3))
    #plt.imshow(tmp, cmap='gray')
    #plt.show()
    tmp = skeletonize(np.clip(tmp, 0, 1)).astype(np.uint8)
    tmp, _, _ = prune(skel_img=tmp, size=int(75))
    #plt.imshow(tmp, cmap='gray')
    #plt.show()
    if circles is not None:
       circles = np.uint16(np.around(circles))
    
       for i in circles[0, :]:
           cv.circle(tmp, (i[0],i[1]), 2 * i[2], 1, -1)
    #plt.imshow(tmp, cmap='gray')
    #plt.show()
    

    # ==НАХОЖДЕНИЕ СТЕПЕНЕЙ ВЕРШИН==

    # массив, где будем хранить посещенные вершины
    visited = np.zeros(tmp.shape, np.uint8) 
    width,height = tmp.shape[1],tmp.shape[0] # ширина и высота изображения

    # массив, в котором k-я компонента есть число вершин степени k в представленном графе
    ans = np.zeros(10, np.uint8) 

    # если нашли вершины, то
    if circles is not None:
       circles = np.uint16(np.around(circles))

       # текущее "время". Нужно, чтобы мы не могли ходить по одним и тем же пикселям в вызове одного dfs'а, но могли ходить по одним и тем же пикселям в разных dfs'ах
       time = 1

       #идем по всем врешинам, считаем их степени в dfs
       for i in circles[0, :]:
           # начинаем поиск в глубину от центра вершины. Функция возвращает степень текущей вершины
           cnt = dfs(time, i, tmp,visited, height, width)
           ans[cnt]+=1 # обновляем ответ
           time += 1 # обновляем "время"
    return ans[3:5]

In [4]:
arr = glob.glob('images/Intermediate/*.jpg')
graph_class = {'[3 3]': [], 
                '[4 1]': [], 
                '[5 2]': [], 
                '[4 2]': []} # словарь для классов графа
for el in arr:
    print('processing of image ' + el)

    src = cv.imread(el) # считываем изображение

    # находим массив со всеми степенями вершин
    ans = get_description_graph(src) 

    # для использования массива как ключа в словаре, переводим его в сторку
    ans_str = str(ans) 
    if ans_str not in graph_class:
        graph_class[ans_str] = list() # создаем новый список под новый класс
    graph_class[ans_str].append(el) # добавляем граф в нужный класс

# вывод ответа
print('\nANSWER:')
num_class = 0
for el in graph_class.values():
    num_class +=1
    print("CLASS", str(num_class) + ":", el)

processing of image images/Intermediate/13.jpg
processing of image images/Intermediate/25.jpg
processing of image images/Intermediate/19.jpg
processing of image images/Intermediate/4.jpg
processing of image images/Intermediate/5.jpg
processing of image images/Intermediate/7.jpg
processing of image images/Intermediate/2.jpg
processing of image images/Intermediate/3.jpg

ANSWER:
CLASS 1: ['images/Intermediate/13.jpg', 'images/Intermediate/2.jpg']
CLASS 2: ['images/Intermediate/25.jpg', 'images/Intermediate/3.jpg']
CLASS 3: ['images/Intermediate/19.jpg', 'images/Intermediate/4.jpg']
CLASS 4: ['images/Intermediate/5.jpg', 'images/Intermediate/7.jpg']
